In [1]:
import os
import tiktoken
import pandas as pd
import numpy as np
import time
from openai import OpenAI
from tqdm import tqdm
from retry import retry
import copy
import random
import json
import jsonlines
import requests

In [2]:
train_file = './Data/FineTuning/train_log4j.tsv'
train_inputs = pd.read_csv(train_file, sep='\t', header=None)[0].tolist()
train_targets = pd.read_csv(train_file, sep='\t', header=None)[1].tolist()

test_file = './Data/FineTuning/test_log4j.tsv'
test_inputs = pd.read_csv(test_file, sep='\t', header=None)[0].tolist()
test_targets = pd.read_csv(test_file, sep='\t', header=None)[1].tolist()

# Cost Estimation

In [24]:
encoding = tiktoken.get_encoding("cl100k_base")

simple_prompt = "You will be provided with a Java method. Your task is to inject at least one logging statement at a " \
                    "rational logging point."
role_prompt = "You are a logging statement generator for Java. " \
                  "You will be provided with a Java method as input. " \
                  "Your task is to inject at least one logging statement at a rational position. " \
                  "The output must be a completed Java method."
example_input = train_inputs[39358]
example_output = train_targets[39358]
instruction_prompt = "Please Analyze the following provided code in Java. " \
                        "Generate at least one logging statements and inject it to the provided Code. " \
                        "Logging statement consist of logging level, logging message, and logging variable. " \
                        "Here are an example: " \
                        "The example input is " + example_input + ", " \
                        "Ths example output is " + example_output + "." \
                        "The output must be a completed Java method. " 

simple_prompt_tokens = encoding.encode(simple_prompt)
role_prompt_tokens = encoding.encode(role_prompt)
instruction_prompt_tokens = encoding.encode(instruction_prompt)

total_tokens_input = list()
total_tokens_output = list()
for idx in tqdm(range(len(inputs))):
    input_tokens = encoding.encode(inputs[idx])
    output_tokens = encoding.encode(inputs[idx])

    total_tokens_input.append(len(input_tokens))
    total_tokens_output.append(len(output_tokens))
    
num_input_tokens_sp = len(inputs)*len(simple_prompt_tokens) + sum(total_tokens_input)
num_input_tokens_role = len(inputs)*len(role_prompt_tokens) + sum(total_tokens_input)
num_input_tokens_inst = len(inputs)*len(instruction_prompt_tokens) + sum(total_tokens_input)

print('Using Simple Prompt:')
print('cost of GPT-4:', (((0.03/1000) * num_input_tokens_sp) + ((0.06/1000) * sum(total_tokens_output))))
print('cost of GPT-4-Turbo:', (((0.01/1000) * num_input_tokens_sp) + ((0.03/1000) * sum(total_tokens_output))))
print('cost of GPT-4o:', (((0.005/1000) * num_input_tokens_sp) + ((0.015/1000) * sum(total_tokens_output))))
print('cost of GPT-3.5-Turbo:', (((0.0005/1000) * num_input_tokens_sp) + ((0.0015/1000) * sum(total_tokens_output))))
print('\n')

print('Using Role Prompt:')
print('cost of GPT-4:', (((0.03/1000) * num_input_tokens_role) + ((0.06/1000) * sum(total_tokens_output))))
print('cost of GPT-4-Turbo:', (((0.01/1000) * num_input_tokens_role) + ((0.03/1000) * sum(total_tokens_output))))
print('cost of GPT-4o:', (((0.005/1000) * num_input_tokens_role) + ((0.015/1000) * sum(total_tokens_output))))
print('cost of GPT-3.5-Turbo:', (((0.0005/1000) * num_input_tokens_role) + ((0.0015/1000) * sum(total_tokens_output))))
print('\n')

print('Using Instruction Prompt:')
print('cost of GPT-4:', (((0.03/1000) * num_input_tokens_inst) + ((0.06/1000) * sum(total_tokens_output))))
print('cost of GPT-4-Turbo:', (((0.01/1000) * num_input_tokens_inst) + ((0.03/1000) * sum(total_tokens_output))))
print('cost of GPT-4o:', (((0.005/1000) * num_input_tokens_inst) + ((0.015/1000) * sum(total_tokens_output))))
print('cost of GPT-3.5-Turbo:', (((0.0005/1000) * num_input_tokens_inst) + ((0.0015/1000) * sum(total_tokens_output))))
print('cost of GPT-3.5-Turbo (legacy):', (((0.0015/1000) * num_input_tokens_inst) + ((0.0040/1000) * sum(total_tokens_output))))
print('\n')

100%|██████████| 6811/6811 [00:01<00:00, 6419.45it/s]

Using Simple Prompt:
cost of GPT-4: 139.48959
cost of GPT-4-Turbo: 61.42779
cost of GPT-4o: 30.713895
cost of GPT-3.5-Turbo: 3.0713895000000004


Using Role Prompt:
cost of GPT-4: 143.37186
cost of GPT-4-Turbo: 62.72188
cost of GPT-4o: 31.36094
cost of GPT-3.5-Turbo: 3.1360940000000004


Using Instruction Prompt:
cost of GPT-4: 236.75066999999999
cost of GPT-4-Turbo: 93.84815
cost of GPT-4o: 46.924075
cost of GPT-3.5-Turbo: 4.6924075
cost of GPT-3.5-Turbo (legacy): 13.3306595




# Batch Dataset Construction

In [ ]:
simple_prompt = "You will be provided with a Java method. Your task is to inject at least one logging statement at a " \
                    "rational logging point."
role_prompt = "You are a logging statement generator for Java. " \
                  "You will be provided with a Java method as input. " \
                  "Your task is to inject at least one logging statement at a rational position. " \
                  "The output must be a completed Java method."
example_input = train_inputs[39358]
example_output = train_targets[39358]
instruction_prompt = "Please Analyze the following provided code in Java. " \
                        "Generate at least one logging statement and inject it to the provided code. " \
                        "Logging statement is embedded in source code to understand system behavior, monitoring choke-points and debugging. " \
                        "A logging statement consist of logging level, logging message, and/or logging variable. " \
                        "The output must be a completed Java method. \n" \
                        "Here are an example:\n" \
                        "```The example input is\n" + example_input + "\n```\n" \
                        "```The example output is\n" + example_output + "\n```\n" \
                        "In this example, the generated logging statement for input is " + '''\'LOG . info ( "Received node: " + node . getIdentity ( ) + " status: " + node . getStatus ( ) + " type: " + node . getType ( ) ) ;\'. ''' + "The generated logging statement is injected in the FOR loop \'for ( CssNode node : nodes ) { }\'."

prompt_type = {"simple": simple_prompt, 
                "role": role_prompt,
                "instruction": instruction_prompt}

In [20]:
max_tokens = 512
model = "gpt-3.5-turbo-0613"

batch = []
for i in range(len(test_inputs)):
    custom_id = "request-" + str(i+1)
    prompt = prompt_type["instruction"]
    request = {"custom_id": custom_id, 
               "method": "POST", "url": "/v1/chat/completions", 
               "body": {"model": model,
                        "messages":[{"role": "system", "content": prompt},{"role": "user", "content": test_inputs[i]}], 
                        "max_tokens": max_tokens}}
    batch.append(request)

def save_list_as_jsonl(file_path, data):
    with open(file_path, 'w') as f:
        for item in data:
            json.dump(item, f)
            f.write('\n')

# save_list_as_jsonl('./gpt-3.5-turbo_simple_prompt_inputs.jsonl', batch)
# save_list_as_jsonl('./gpt-3.5-turbo_role_prompt_inputs.jsonl', batch)
save_list_as_jsonl('./gpt-3.5-turbo_instruction_prompt_inputs.jsonl', batch)

# GPT-3.5-Turbo Batch Inference

### simple prompt version

In [35]:
from openai import OpenAI
api_key = ''
client = OpenAI(api_key = api_key)

In [36]:
batch_input_file = client.files.create(
  file=open("simple_prompt_inputs.jsonl", "rb"),
  purpose="batch"
)

In [39]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "Evaluation of GPT-3.5-Turbo with using simple prompt"
    }
)

Batch(id='batch_qAIvekErAMqji8fI4hSnOlri', completion_window='24h', created_at=1716293009, endpoint='/v1/chat/completions', input_file_id='file-SRYuSn4Dy9omkRYJSYy4uIo3', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1716379409, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Evaluation of GPT-4o with using simple prompt'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [42]:
client.batches.retrieve('')

Batch(id='batch_qAIvekErAMqji8fI4hSnOlri', completion_window='24h', created_at=1716293009, endpoint='/v1/chat/completions', input_file_id='file-SRYuSn4Dy9omkRYJSYy4uIo3', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1716298372, error_file_id=None, errors=None, expired_at=None, expires_at=1716379409, failed_at=None, finalizing_at=1716297920, in_progress_at=1716293011, metadata={'description': 'Evaluation of GPT-4o with using simple prompt'}, output_file_id='file-UYWoKFVlkUeVXqDgmAZNPENp', request_counts=BatchRequestCounts(completed=6811, failed=0, total=6811))

In [102]:
content = client.files.content('')
simple_outputs = []
for message in content.iter_lines():
    simple_outputs.append(message)


def save_list_as_jsonl(file_path, data):
    with open(file_path, 'w') as f:
        for item in data:
            json.dump(item, f)
            f.write('\n')
            
save_list_as_jsonl('./Data/PredictionResult/gpt-3.5-turbo_simple-prompt_output.jsonl', simple_outputs)

### role prompt version

In [103]:
from openai import OpenAI
api_key = ''
client = OpenAI(api_key = api_key)

In [104]:
batch_input_file = client.files.create(
  file=open("role_prompt_inputs.jsonl", "rb"),
  purpose="batch"
)

In [105]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "Evaluation of GPT-3.5-Turbo with using role prompt"
    }
)

Batch(id='batch_WR3K77Wx769tVsEbl8yXbgRO', completion_window='24h', created_at=1716304788, endpoint='/v1/chat/completions', input_file_id='file-ieK0RcKpfshB1YseGrcDqU8I', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1716391188, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Evaluation of GPT-4o with using role prompt'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [107]:
client.batches.retrieve('')

Batch(id='batch_WR3K77Wx769tVsEbl8yXbgRO', completion_window='24h', created_at=1716304788, endpoint='/v1/chat/completions', input_file_id='file-ieK0RcKpfshB1YseGrcDqU8I', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1716319320, error_file_id=None, errors=None, expired_at=None, expires_at=1716391188, failed_at=None, finalizing_at=1716318808, in_progress_at=1716304790, metadata={'description': 'Evaluation of GPT-4o with using role prompt'}, output_file_id='file-hvd8cFGOLEu9dC4AiYPojfRF', request_counts=BatchRequestCounts(completed=6811, failed=0, total=6811))

In [111]:
content = client.files.content('')
role_outputs = []
for message in content.iter_lines():
    role_outputs.append(message)


def save_list_as_jsonl(file_path, data):
    with open(file_path, 'w') as f:
        for item in data:
            json.dump(item, f)
            f.write('\n')
            
save_list_as_jsonl('./Data/PredictionResult/gpt-3.5-turbo_role-prompt_output.jsonl', role_outputs)

### instruction prompt version

In [21]:
from openai import OpenAI
api_key = ''
client = OpenAI(api_key = api_key)

In [22]:
batch_input_file = client.files.create(
  file=open("gpt-3.5-turbo_instruction_prompt_inputs.jsonl", "rb"),
  purpose="batch"
)

In [23]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "Evaluation of GPT-3.5-Turbo with using instruction prompt"
    }
)

Batch(id='batch_nyBi78MYjTIn3UvJjxwN9bz0', completion_window='24h', created_at=1716556131, endpoint='/v1/chat/completions', input_file_id='file-Xuu8fBFUJA09kN9WkiqNJLN8', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1716642531, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Evaluation of GPT-3.5-turbo-0613 with using instruction prompt'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [27]:
client.batches.retrieve('')

Batch(id='batch_nyBi78MYjTIn3UvJjxwN9bz0', completion_window='24h', created_at=1716556131, endpoint='/v1/chat/completions', input_file_id='file-Xuu8fBFUJA09kN9WkiqNJLN8', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1716560134, error_file_id=None, errors=None, expired_at=None, expires_at=1716642531, failed_at=None, finalizing_at=1716559660, in_progress_at=1716556133, metadata={'description': 'Evaluation of GPT-3.5-turbo-0613 with using instruction prompt'}, output_file_id='file-9f8MFWPobdVdWjCRhBryuiBU', request_counts=BatchRequestCounts(completed=6811, failed=0, total=6811))

In [28]:
content = client.files.content('')
instruction_outputs = []
for message in content.iter_lines():
    instruction_outputs.append(message)

def save_list_as_jsonl(file_path, data):
    with open(file_path, 'w') as f:
        for item in data:
            json.dump(item, f)
            f.write('\n')
            
save_list_as_jsonl('./Data/PredictionResult/gpt-3.5-turbo_instruction-prompt_output.jsonl', instruction_outputs)